In [13]:
from glob import glob
import simplejson as json
import gzip
import pandas as pd

from polyglot.text import Text
from polyglot.downloader import downloader

In [4]:
speeches_2016  = pd.read_csv('/notebooks/datasets/speeches/2016.csv_all.csv')

In [7]:
candidates = ['Hillary Clinton', 'Bernie Sanders', 'Donald Trump']

In [6]:
speeches_2016.Actor.unique()

array(['Hillary Clinton', 'Bernie Sanders', "Martin O'Malley",
       'Lincoln Chafee', 'Jim Webb', 'Donald Trump', 'John Kasich',
       'Ted Cruz', 'Marco Rubio', 'Ben Carson', 'Jeb Bush',
       'Chris Christie', 'Carly Fiorina', 'Rick Santorum', 'Rand Paul',
       'Mike Huckabee', 'George Pataki', 'Lindsey Graham', 'Bobby Jindal',
       'Scott Walker', 'Rick Perry'], dtype=object)

In [29]:
def get_entity_sentiment(e):
    try:
        pos=e.positive_sentiment
        neg=e.negative_sentiment
        return pos,neg
    except:
        return '',''
def get_sentiment(t):
    blob = TextBlob(t.decode('utf-8'))
    return map(lambda s:s.sentiment.polarity,blob.sentences)
def get_noun_phrases(t):
    blob = TextBlob(t.decode('utf-8'))
    return list(blob.noun_phrases)
def map_entity(e):
    pos,neg=get_entity_sentiment(e)
    return {'tag':e.tag,'entity':" ".join(e),'pos':pos,'neg':neg}
def get_entites(text):
    try:
        polyglot_text=Text(text)
        entities=polyglot_text.entities
        return map(map_entity,entities)
    except:
        return []
     
def map_to_data(tweet):
    user_name=tweet['user']['name']
    text=tweet['text']
    return {'text':text,'user':user_name,'entities':get_entites(text),'sentiment':get_sentiment(text),'noun_phrases':get_noun_phrases(text)}

In [14]:
def get_sentiment(e):
    try:
        pos=e.positive_sentiment
        neg=e.negative_sentiment
        return pos,neg
    except:
        return '',''

def map_entity(e):
    pos,neg=get_sentiment(e)
    return {'tag':e.tag,'entity':" ".join(e),'pos':pos,'neg':neg}
def get_entites(text):
    try:
        polyglot_text=Text(text)
        entities=polyglot_text.entities
        return map(map_entity,entities)
    except:
        return []
     
def map_to_data(tweet):
    user_name=tweet['user']['name']
    text=tweet['text']
    return {'text':text,'user':user_name,'entities':get_entites(text)}

In [11]:
speeches_2016 = speeches_2016[speeches_2016.Actor.isin(candidates)]
speeches_2016.head(2)

,Unnamed: 0,url,Date,Title,Link,Actor,Texts
0,0,http://www.presidency.ucsb.edu/2016_election_s...,"April 12, 2015",Video Remarks Announcing Candidacy for President,http://www.presidency.ucsb.edu/ws/index.php?pi...,Hillary Clinton,Unidentified Female: I'm getting ready for a l...
1,1,http://www.presidency.ucsb.edu/2016_election_s...,"April 14, 2015",Remarks in a Question and Answer Session at Ki...,http://www.presidency.ucsb.edu/ws/index.php?pi...,Hillary Clinton,[joined in progress]\n[unknown student speaker...


In [15]:
speeches_2016['entities']=speeches_2016['Texts'].map(get_entites)

In [40]:
pd.read_csv('/notebooks/datasets/speeches/2008_all.csv').shape

(1, 7)

In [36]:
speeches_2016.to_pickle('/notebooks/datasets/speeches/2016_all_entities.pkl')

In [37]:
speeches_2016.columns

Index([u'Unnamed: 0', u'url', u'Date', u'Title', u'Link', u'Actor', u'Texts',
       u'entities'],
      dtype='object')

In [35]:
speeches_2016.ix[0]['entities']

[{'entity': u"I'm", 'neg': 0, 'pos': 1.0, 'tag': u'I-PER'},
 {'entity': u"I'm", 'neg': 0, 'pos': 0.82736842105263153, 'tag': u'I-PER'},
 {'entity': u"I'm", 'neg': 0, 'pos': 0.78526315789473689, 'tag': u'I-PER'},
 {'entity': u"we've", 'neg': 0, 'pos': 0.76421052631578945, 'tag': u'I-PER'},
 {'entity': u"I've", 'neg': 0, 'pos': 0.71157894736842109, 'tag': u'I-PER'},
 {'entity': u'Clinton', 'neg': 0, 'pos': 0.66736842105263161, 'tag': u'I-PER'},
 {'entity': u"I'm", 'neg': 0, 'pos': 0.66526315789473678, 'tag': u'I-PER'},
 {'entity': u'America', 'neg': 0, 'pos': 0.58526315789473682, 'tag': u'I-LOC'},
 {'entity': u'Hillary Clinton',
  'neg': 0,
  'pos': 0.55368421052631578,
  'tag': u'I-PER'},
 {'entity': u'Gerhard Peters',
  'neg': 0,
  'pos': 0.53368421052631576,
  'tag': u'I-PER'},
 {'entity': u'John T', 'neg': 0, 'pos': 0.53052631578947373, 'tag': u'I-PER'},
 {'entity': u'Woolley', 'neg': 0, 'pos': 0.5273684210526316, 'tag': u'I-PER'},
 {'entity': u'American',
  'neg': 0,
  'pos': 0.5242

In [32]:
speeches_2016.ix[0]['Link']

'http://www.presidency.ucsb.edu/ws/index.php?pid=110028'

In [21]:
cands_map = {'Hillary': 'Hillary Clinton', 'Hillary Clinton': 'Hillary Clinton', 'Clinton': 'Hillary Clinton', 
            'Donald Trump': 'Donald Trump', 'Donald': 'Donald Trump', 'Trump': 'Donald Trump',
            'Bernie Sanders': 'Bernie Sanders', 'Bernie': 'Bernie Sanders', 'Sanders': 'Bernie Sanders'}

def check_if_candidate(x):
    if x in cands_map:
        return cands_map[x]
    else:
        return None

In [26]:
tweets = speeches_2016
candidate_col = 'Actor'
candidates = []
for row_id, tweet in tweets.iterrows():
    cand = tweet[candidate_col]
    for ent in tweet['entities']:
        e = ent['entity']
        if check_if_candidate(e):
            candidates.append((cand, check_if_candidate(e)))
[x for x in candidates if x[0] != x[1]]

In [24]:
len(candidates)

9081